BD SQL & NOSQL Project
======================
### Project based on data from RGPH (recensement général de la population et de l’habitat) in 2014
### DATA SOURCE: [RGPH Website](http://rgphentableaux.hcp.ma)

## Part 03: Migrate From MySQL to MongoDB Database

In [ ]:
# Install dependencies
!pip install pymysql pymongo

In [ ]:
# Import libraries
import pymysql
import pymongo

### Question 01: MongoDB Database Conception
![MongoDB DB Diagram](https://i.imgur.com/jVPgLKR.png)

In [ ]:
#Define Databases Connections

# MySql Database Connection
MYSQL_PARAMS = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': 'ehtp',
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}
mysql_cnx = pymysql.connect(**MYSQL_PARAMS)
mysql_crs = mysql_cnx.cursor()

# MongoDB Database Connection
MONGODB_PARAMS = {
    'host': 'localhost',
    'port': 27017,
    'username': 'ehtp',
    'password': 'ehtp'
}
mongodb_clt = pymongo.MongoClient(**MONGODB_PARAMS)


In [ ]:
#Creating and Filling Tables

sql_query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'ehtp_rgph'
    ORDER BY table_rows
"""
mysql_crs.execute(sql_query)
mysql_tables = mysql_crs.fetchall()

sql_query = "USE ehtp_rgph;"
mysql_crs.execute(sql_query)

mongodb_clt.drop_database('ehtp_rgph')
mongodb_db = mongodb_clt.ehtp_rgph
for table in mysql_tables:
    sql_query = f"SELECT * FROM {table['table_name']}"
    mysql_crs.execute(sql_query)
    table_data = []
    for row in mysql_crs.fetchall():
        row['_id'] = row.pop('code', None) or row.pop('id')
        row.pop('id', None)
        table_data.append(row)

    mongodb_table = mongodb_db[table['table_name']]

    new_result = mongodb_table.insert_many(
        table_data
    )